In [1]:
import transformers
import torch
from torch import nn
import numpy as np

In [2]:
context_type='bert-base-uncased'
max_length = 256

In [3]:
config = transformers.BertConfig.from_pretrained('bert-base-uncased')  
# bert = transformers.BertModel.from_pretrained(context_type, config=config)
# bert = bert.to('cuda:0')
tokenizer = transformers.BertTokenizer.from_pretrained(context_type)

In [19]:
class MyBert(nn.Module):
    def __init__(self, context_type, max_length):
        super(MyBert, self).__init__()
        config = transformers.BertConfig.from_pretrained('bert-base-uncased')  
        bert = transformers.BertModel.from_pretrained(context_type, config=config)
        tokenizer = transformers.BertTokenizer.from_pretrained(context_type)
        
    def forward(self, x):
        x = tokenizer(
            text=tmp_text, 
            return_tensors="pt", 
            truncation='longest_first',
            padding='max_length',
            max_length=max_length,
        )
        out = bert(**x, return_dict=True)
        return out.last_hidden_state

In [20]:
my_bert = MyBert(context_type, max_length)

In [4]:
tmp_text = 'guideline hand hygiene health care setting recommendation healthcare infection control practice advisory committee hicpac shea ape idsa hand hygiene task force prepared john boyce md didier pittet md hospital saint raphael new haven connecticut university geneva geneva switzerland material report originate national center infectious disease james hughes md director division healthcare quality promotion steve solomon md acting director summary guideline hand hygiene health care setting health care worker hcw review data regarding handwash hand antisepsi health care setting addition specific recommendation promote improve hand hygiene practice reduce transmission pathogenic microorganism patient personnel health care setting report review study publish cdc guideline garner js favero cdc guideline handwash hospital environmental control infect control ape guideline larson el ape guidelines committee ape guideline handwash hand antisepsi health care setting infect control issue depth review hand hygiene practice hcw level adherence personnel recommend handwash practice factor adverse affecting adherence new study vivo efficacy alcohol base hand rub low incidence dermatitis associate use review recent study demonstrate value multidisciplinary hand hygiene promotion program potential role alcohol base hand rub improve hand hygiene practice summarize recommendation concerning related issue e use surgical hand antiseptic hand lotion cream wearing artificial fingernail part review scientific data regarding hand hygiene guideline hand hygiene health care setting recommendation healthcare infection control practice advisory committee hicpac shea ape idsa hand hygiene task force prepared john boyce md didier pittet md hospital saint raphael new haven connecticut university geneva geneva switzerland material report originate national center infectious disease james hughes md director division healthcare quality promotion steve'

In [16]:
dummy_input = tokenizer(
    text=tmp_text, 
    return_tensors="pt", 
    truncation='longest_first',
    padding='max_length',
    max_length=max_length,
)
dummy_input = dummy_input.to(bert.device)

In [17]:
dummy_input = (dummy_input['input_ids'], dummy_input['token_type_ids'], dummy_input['attention_mask'])

In [7]:
dummy_input

{'input_ids': tensor([[  101,  5009,  4179,  2192, 19548,  2740,  2729,  4292, 12832,  9871,
          8985,  2491,  3218,  7319,  2837,  7632, 21906,  6305, 16994, 23957,
          8909,  3736,  2192, 19548,  4708,  2486,  4810,  2198,  2879,  3401,
          9108,  2106,  3771, 15091,  3388,  9108,  2902,  3002, 12551,  2047,
          4033,  6117,  2118,  9810,  9810,  5288,  3430,  3189, 21754,  2120,
          2415, 16514,  4295,  2508,  8099,  9108,  2472,  2407,  9871,  3737,
          4712,  3889,  9168,  9108,  3772,  2472, 12654,  5009,  4179,  2192,
         19548,  2740,  2729,  4292,  2740,  2729,  7309, 16731,  2860,  3319,
          2951,  4953,  2192, 28556,  2192,  3424,  3366,  4523,  2072,  2740,
          2729,  4292,  2804,  3563, 12832,  5326,  5335,  2192, 19548,  3218,
          5547,  6726, 26835,  2594, 12702, 21759,  7088,  6491,  5776,  5073,
          2740,  2729,  4292,  3189,  3319,  2817, 10172, 26629,  5009,  4179,
         18661,  1046,  2015,  6904,  

In [25]:
torch.onnx.export(bert, dummy_input, "./bert.onnx", opset_version=10)

In [1]:
import torch.onnx
import torchvision

# Standard ImageNet input - 3 channels, 224x224,
# values don't matter as we care about network structure.
# But they can also be real inputs.
dummy_input = torch.randn(1, 3, 224, 224)
# Obtain your model, it can be also constructed in your script explicitly
model = torchvision.models.alexnet(pretrained=True)
# Invoke export
torch.onnx.export(model, dummy_input, "alexnet.onnx")

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /home/ninglu_shao/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


In [12]:
from pathlib import Path
from transformers.convert_graph_to_onnx import convert

# Handles all the above steps for you
convert(framework="pt", model="bert-base-uncased", output=Path('onnx/a.onnx'), opset=11)

ONNX opset version set to: 11
Loading pipeline (model: bert-base-uncased, tokenizer: bert-base-uncased)
Using framework PyTorch: 1.8.0
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input token_type_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch', 1: 'sequence'}
Found output output_1 with shape: {0: 'batch'}
Ensuring inputs are in correct order
position_ids is not present in the generated input list.
Generated inputs order: ['input_ids', 'attention_mask', 'token_type_ids']


/home/lin_bo/anaconda3/envs/torch_test/lib/python3.8/site-packages/transformers/modeling_utils.py:1669: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  input_tensor.shape == tensor_shape for input_tensor in input_tensors


In [9]:
from os import environ
from psutil import cpu_count

# Constants from the performance optimization available in onnxruntime
# It needs to be done before importing onnxruntime
environ["OMP_NUM_THREADS"] = str(cpu_count(logical=True)) # OMP 的线程数
environ["OMP_WAIT_POLICY"] = 'ACTIVE'

from onnxruntime import GraphOptimizationLevel, InferenceSession, SessionOptions, get_all_providers


def create_model_for_provider(model_path: str, provider: str) -> InferenceSession: 
  
    assert provider in get_all_providers(), f"provider {provider} not found, {get_all_providers()}"

    # Few properties that might have an impact on performances (provided by MS)
    options = SessionOptions()
    options.intra_op_num_threads = 1
    options.graph_optimization_level = GraphOptimizationLevel.ORT_ENABLE_ALL

  # Load the model as a graph and prepare the CPU backend 
    session = InferenceSession(model_path, options, providers=[provider])
    session.disable_fallback()
    
    return session

In [2]:
import onnxruntime

OSError: libcublas.so.10.0: cannot open shared object file: No such file or directory

In [ ]:
model_path = r'onnx/a.onnx'
options = onnxruntime.SessionOptions()
session = onnxruntime.InferenceSession(model_path, options)
model_inputs = tokenizer(tmp_text, 
                         return_tensors="pt",
                         truncation='longest_first',
                         padding='max_length',
                         max_length=max_length,)
inputs_onnx = {k: v.cpu().detach().numpy() for k, v in model_inputs.items()}
sequence, pooled = session.run(None, inputs_onnx)

In [52]:
cpu_model = create_model_for_provider("onnx/a.onnx", "CPUExecutionProvider") # 使用 优化过的 onnx

# Inputs are provided through numpy array
model_inputs = tokenizer(tmp_text, 
                         return_tensors="pt",
                         truncation='longest_first',
                         padding='max_length',
                         max_length=max_length,)
inputs_onnx = {k: v.cpu().detach().numpy() for k, v in model_inputs.items()}

# Run the model (None = get all the outputs)
sequence, pooled = cpu_model.run(None, inputs_onnx)

# Print information about outputs
print(f"Sequence output: {sequence.shape}, Pooled output: {pooled.shape}")

Sequence output: (1, 256, 768), Pooled output: (1, 768)


In [53]:
pooled

array([[-0.7876456 , -0.37514284, -0.62177825,  0.51467806,  0.04451199,
        -0.06266444,  0.542937  ,  0.32615924, -0.22711486, -0.999966  ,
        -0.0253908 ,  0.66451746,  0.9587116 ,  0.26778808,  0.7770741 ,
        -0.46493056,  0.32824275, -0.47594348,  0.21665512,  0.16078912,
         0.49847522,  0.9990304 ,  0.21279846,  0.41808248,  0.33688176,
         0.5829688 , -0.40536615,  0.78604555,  0.87679315,  0.65466446,
        -0.45456868,  0.34951618, -0.951446  , -0.15150009, -0.6541763 ,
        -0.98015136,  0.21875226, -0.53091437, -0.03381576,  0.00569992,
        -0.6860586 ,  0.34882244,  0.9997967 , -0.20320052,  0.11019732,
        -0.26156852, -0.9999467 ,  0.20039761, -0.67345405,  0.43705562,
         0.2182735 ,  0.46409705,  0.14355789,  0.29323584,  0.33562776,
         0.09232467, -0.10981961,  0.08744046, -0.2797671 , -0.46606663,
        -0.578237  ,  0.4825899 , -0.5072    , -0.8069033 ,  0.03463387,
         0.33685806, -0.21042414, -0.33315817, -0.0

In [12]:
%%time
for i in range(1):
    out = cpu_model.run(None, inputs_onnx)

CPU times: user 44.7 s, sys: 208 ms, total: 44.9 s
Wall time: 44.9 s


In [4]:
model_inputs = tokenizer("A dog is running on beach", 
                         return_tensors="pt",
                         truncation='longest_first',
                         padding='max_length',
                         max_length=max_length,)
model_inputs = model_inputs.to(bert.device)

In [27]:
model_inputs = tokenizer("A dog is running on beach", 
                         return_tensors="pt")
model_inputs = model_inputs.to(bert.device)

In [36]:
model_inputs = tokenizer(tmp_text, 
                         return_tensors="pt",
                         truncation='longest_first',
                         padding='max_length',
                         max_length=max_length,)
model_inputs = model_inputs.to(bert.device)

In [16]:
model_inputs['input_ids'].shape

torch.Size([1, 8])

In [42]:
model_inputs

{'input_ids': tensor([[  101,  5009,  4179,  2192, 19548,  2740,  2729,  4292, 12832,  9871,
          8985,  2491,  3218,  7319,  2837,  7632, 21906,  6305, 16994, 23957,
          8909,  3736,  2192, 19548,  4708,  2486,  4810,  2198,  2879,  3401,
          9108,  2106,  3771, 15091,  3388,  9108,  2902,  3002, 12551,  2047,
          4033,  6117,  2118,  9810,  9810,  5288,  3430,  3189, 21754,  2120,
          2415, 16514,  4295,  2508,  8099,  9108,  2472,  2407,  9871,  3737,
          4712,  3889,  9168,  9108,  3772,  2472, 12654,  5009,  4179,  2192,
         19548,  2740,  2729,  4292,  2740,  2729,  7309, 16731,  2860,  3319,
          2951,  4953,  2192, 28556,  2192,  3424,  3366,  4523,  2072,  2740,
          2729,  4292,  2804,  3563, 12832,  5326,  5335,  2192, 19548,  3218,
          5547,  6726, 26835,  2594, 12702, 21759,  7088,  6491,  5776,  5073,
          2740,  2729,  4292,  3189,  3319,  2817, 10172, 26629,  5009,  4179,
         18661,  1046,  2015,  6904,  

In [11]:
model_inputs['input_ids'] = model_inputs['input_ids'].int()
model_inputs['token_type_ids'] = model_inputs['token_type_ids'].int()
model_inputs['attention_mask'] = model_inputs['attention_mask'].int()

In [12]:
model_inputs['input_ids'].type()

'torch.IntTensor'

In [9]:
inputs_onnx = {k: v.cpu().detach().numpy() for k, v in model_inputs.items()}
inputs_onnx

{'input_ids': array([[ 101, 1037, 3899, 2003, 2770, 2006, 3509,  102]]),
 'token_type_ids': array([[0, 0, 0, 0, 0, 0, 0, 0]]),
 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [13]:
sequence

array([[[-0.45129588,  0.17464024, -0.32815534, ..., -0.34352878,
          0.95904636, -0.3726815 ],
        [-0.02382827,  0.18869871, -0.14093272, ..., -0.22917065,
          1.0786096 , -0.26578447],
        [-0.3177335 ,  0.67381006,  0.25352606, ..., -0.2675582 ,
          0.41499317, -0.7243721 ],
        ...,
        [-0.10590474,  0.263191  ,  0.12382512, ..., -0.5368702 ,
          0.45611474, -0.33608583],
        [ 0.21408592, -0.13089575,  0.04547495, ..., -0.42025897,
          0.41084263, -0.9893264 ],
        [ 0.16522332,  0.10745797, -0.46665922, ...,  0.02295056,
         -0.15455595, -0.34739056]]], dtype=float32)

In [37]:
%%time
for i in range(1):  
    out = bert(**model_inputs)

CPU times: user 3.02 s, sys: 979 ms, total: 4 s
Wall time: 220 ms


In [41]:
%%time
for i in range(1000):  
    out = bert(**model_inputs)

CPU times: user 42min 41s, sys: 29.9 s, total: 43min 11s
Wall time: 2min 12s


In [57]:
for i in range(256):
    print(model_inputs['input_ids'][0][i].item(), ',', end='')

101 ,1037 ,3899 ,2003 ,2770 ,2006 ,3509 ,102 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,

In [61]:
out

(tensor([[[-0.3401,  0.4414, -0.0335,  ..., -0.3966,  0.1183,  0.2993],
          [ 0.4137, -0.2426,  0.7700,  ..., -0.0194,  0.4660, -0.0095],
          [-0.4527, -0.0234,  0.5384,  ..., -0.3647, -0.3127, -1.1680],
          ...,
          [-0.1401, -0.5340,  1.0292,  ..., -0.3733, -0.3382, -0.3510],
          [ 0.1625,  0.1707, -0.0268,  ..., -0.6657, -0.6873, -0.4199],
          [ 0.3701,  0.4741, -0.3531,  ..., -0.0108, -0.0713,  0.1101]]],
        grad_fn=<NativeLayerNormBackward>),
 tensor([[-0.7876, -0.3751, -0.6218,  0.5147,  0.0445, -0.0627,  0.5429,  0.3262,
          -0.2271, -1.0000, -0.0254,  0.6645,  0.9587,  0.2678,  0.7771, -0.4649,
           0.3282, -0.4759,  0.2167,  0.1608,  0.4985,  0.9990,  0.2128,  0.4181,
           0.3369,  0.5830, -0.4054,  0.7860,  0.8768,  0.6547, -0.4546,  0.3495,
          -0.9514, -0.1515, -0.6542, -0.9802,  0.2188, -0.5309, -0.0338,  0.0057,
          -0.6861,  0.3488,  0.9998, -0.2032,  0.1102, -0.2616, -0.9999,  0.2004,
          -0.67